In [5]:
import numpy as np

def cart2sph(x,y,z,vx,vy,vz):
	r = np.sqrt(x**2+y**2+z**2)
	theta = np.arccos(z/r)
	phi = np.sign(y)*np.arccos(x/np.sqrt(x**2+y**2))
	vr = (x*vx + y*vy + z*vz)/r
	vtheta = np.sum(np.array([vx,vy,vz])*np.array([np.cos(theta)*np.cos(phi),np.cos(theta)*np.sin(phi),-np.sin(theta)]))/r # come back and make these trigonometric functions into functions of x, y, z
	vphi = np.sum(np.array([vx,vy,vz])*np.array([-np.sin(phi),np.cos(phi),0]))/np.sqrt(x**2 + y**2)
	return r,theta,phi,vr,vtheta,vphi

def sph2cart(r,theta,phi,vr,vtheta,vphi=0):
	x = r*np.sin(theta)*np.cos(phi)
	y = r*np.sin(theta)*np.sin(phi)
	z = r*np.cos(theta)
	rho = r*np.sin(theta)
	vx = vr*np.sin(theta)*np.cos(phi) + r*vtheta*np.cos(theta)*np.cos(phi) - rho*vphi*np.sin(phi)
	vy = vr*np.sin(theta)*np.sin(phi) + r*vtheta*np.cos(theta)*np.sin(phi) + rho*vphi*np.cos(phi)
	vz = vr*np.cos(theta) - np.sin(theta)
	return x,y,z,vx,vy,vz

In [6]:
cart2sph(1,0,0,0,0,-1)

(1.0, 1.5707963267948966, 0.0, 0.0, 1.0, 0.0)

In [7]:
sph2cart(1.0, 1.5707963267948966, 0.0, 0.0, 1.0, 0.0)

(1.0, 0.0, 6.123233995736766e-17, 6.123233995736766e-17, 0.0, -1.0)

In [8]:
cart2sph(np.cos(np.pi/4),0,np.sin(np.pi/4),np.cos(np.pi/4),0,-np.sin(np.pi/4))

(1.0, 0.7853981633974483, 0.0, 0.0, 1.0000000000000002, 0.0)

In [9]:
sph2cart(1.0, 0.7853981633974483, 0.0, 0.0, 1.0000000000000002, 0.0)

(0.7071067811865476,
 0.0,
 0.7071067811865476,
 0.7071067811865477,
 0.0,
 -0.7071067811865476)

In [27]:
def cart2sph(x,y,z,vx,vy,vz):
	r = np.sqrt(x**2+y**2+z**2)
	theta = np.arccos(z/r)
	phi = np.sign(y)*np.arccos(x/np.sqrt(x**2+y**2))
	rho = np.sqrt(x**2+y**2)
	sinTheta = rho/r
	cosTheta = z/r
	sinPhi = y/rho
	cosPhi = x/rho
	vr = (x*vx + y*vy + z*vz)/r
	vtheta = np.sum(np.array([vx,vy,vz])*np.array([cosTheta*cosPhi,cosTheta*sinPhi,-sinTheta]))/r # come back and make these trigonometric functions into functions of x, y, z
    vtheta = (vx*cosTheta*cosPhi + vy*cosTheta*sinPhi - vz*sinTheta)/r
	vphi = np.sum(np.array([vx,vy,vz])*np.array([-sinPhi,cosPhi,0]))/rho
    vphi = (-vx*sinPhi + vy*cosPhi)/rho
	return r,theta,phi,vr,vtheta,vphi

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 12)

In [16]:
cart2sph(1,0,0,0,0,-1)

(1.0, 1.5707963267948966, 0.0, 0.0, 1.0, 0.0)

In [17]:
cart2sph(np.cos(np.pi/4),0,np.sin(np.pi/4),np.cos(np.pi/4),0,-np.sin(np.pi/4))

(1.0, 0.7853981633974483, 0.0, 0.0, 1.0000000000000002, 0.0)

In [34]:
def cart2sph(x,y,z,vx,vy,vz):
	r = np.sqrt(x**2+y**2+z**2)
	theta = np.arccos(z/r)
	phi = np.sign(y)*np.arccos(x/np.sqrt(x**2+y**2))
	rho = np.sqrt(x**2+y**2)
	sinTheta = rho/r
	cosTheta = z/r
	sinPhi = y/rho
	cosPhi = x/rho
	vr = (x*vx + y*vy + z*vz)/r
	vtheta = (vx*cosTheta*cosPhi + vy*cosTheta*sinPhi - vz*sinTheta)/r
	vphi = (-vx*sinPhi + vy*cosPhi)/rho
	return r,theta,phi,vr,vtheta,vphi

In [35]:
cart2sph(1,0,0,0,0,-1)

(1.0, 1.5707963267948966, 0.0, 0.0, 1.0, 0.0)

In [36]:
cart2sph(np.cos(np.pi/4),0,np.sin(np.pi/4),np.cos(np.pi/4),0,-np.sin(np.pi/4))

(1.0, 0.7853981633974483, 0.0, 0.0, 1.0000000000000002, 0.0)

In [5]:
# takes in the initial image and prepares it for the lensing simulation.
# saves an array of the RGB values to a file

import numpy as np
import PIL
from PIL import Image as Im

# method to be used by other files in case we don't want to run this individually
def imageTakeInner(image_file):
    image = Im.open(image_file)
    image = np.array(image)
    return image

# method to find where the ray passes through an image
def findPixel(y_center, x_center, z_position, pixel_length, ri, v):
    xi, yi, zi = ri # initial positions, should input the final positions BEFORE the ray passes through the image
    vx, vy, vz = v # velocities from the same time as the initial position
    Z = z_position - zi
    xf = xi + Z*(vx/vz)
    yf = yi + Z*(vy/vz)
    j = (xf + x_center)/pixel_length
    i = (yf + y_center)/pixel_length
    j, i = round(j), round(i)
    return i, j # y then x indices for the pixel

# choose the image to use and the file to save to HERE
image_file = 'stock-photo-young-attractive-dangerous-woman-aiming-at-gold-fish-55445056.jpg' # input file
array_file = 'usable.npy' # 0utput file with the format .npy

image = Im.open(image_file)
image = np.array(image)
np.save(array_file, image)

In [6]:
findPixel(0,0,10,1,np.array([0,0,0]),np.array([1,-1,2]))

(-5, 5)

In [7]:
findPixel(3,8,10,1,np.array([0,0,0]),np.array([1,-1,2]))

(-2, 13)

In [8]:
findPixel(3,8.1,10,1,np.array([0,0,0]),np.array([1,-1,2]))

(-2, 13)